In [1]:
import os
import splitfolders
from PIL import Image
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from pathlib import Path
import PIL

# Renaming the image names


In [36]:
plant_names=["banana", "coconut", "maize", "wheat", "sugarcane"]

In [14]:
for i in range(len(plant_names)): 
    folder_path = r"C:\Users\GREESHMANTH\Desktop\Major Project\Code\Dataset\\"+plant_names[i]
    for count, filename in enumerate(os.listdir(folder_path)):
        extension = filename.split(".")[-1]
        os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, "{}_{}.{}".format(plant_names[i],count, extension)))

# Dataset Splitting

In [11]:
def split_folder(input_folder, output_folder):
    splitfolders.ratio(input_folder, output_folder,seed=41, ratio=(.6,.2,.2))

In [13]:
input_folder=r'C:\Users\GREESHMANTH\Desktop\Major Project\Code\Dataset'
output_folder=r'C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split'
split_folder(input_folder, output_folder)

Copying files: 509 files [00:00, 520.19 files/s]


# Data Preprocessing
1. Noise Removal - fastNlMeansDenoisingColored
2. Normalization - Min-Max normalization
3. Image Enhancement - histogram_equalization

### Noise Removal - fastNlMeansDenoisingColored

In [2]:
def denoise_images(directory):
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            print(file)
            if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):
                filepath = subdir + os.sep + file
                image = cv2.imread(filepath)
                denoised_image = cv2.fastNlMeansDenoisingColored(image, None, 5,5, 7, 21)
                cv2.imwrite(filepath, denoised_image)

In [22]:
denoise_images(r"C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split\train")

banana_1.jpg
banana_10.jpg
banana_13.jpeg
banana_15.png
banana_16.jpg
banana_18.jpg
banana_2.png
banana_22.jpg
banana_23.jpg
banana_25.jpg
banana_26.jpeg
banana_29.jpg
banana_3.jpg
banana_30.jpg
banana_32.jpg
banana_36.jpg
banana_38.jpg
banana_39.jpeg
banana_4.jpg
banana_41.png
banana_42.jpg
banana_43.jpg
banana_44.jpg
banana_45.jpg
banana_46.jpg
banana_47.jpg
banana_48.jpg
banana_5.jpg
banana_50.jpg
banana_51.jpg
banana_53.jpg
banana_54.jpg
banana_55.jpg
banana_56.jpg
banana_59.jpg
banana_6.jpg
banana_61.jpg
banana_62.jpg
banana_64.jpg
banana_65.jpg
banana_66.jpg
banana_68.jpg
banana_69.jpg
banana_7.jpg
banana_70.jpg
banana_72.jpg
banana_74.jpg
banana_75.jpg
banana_76.jpg
banana_78.jpg
banana_8.jpg
banana_80.jpg
banana_87.jpg
banana_9.jpg
coconut_0.jpeg
coconut_11.jpg
coconut_12.jpeg
coconut_17.jpg
coconut_18.jpg
coconut_19.jpg
coconut_2.jpg
coconut_23.jpg
coconut_25.jpg
coconut_26.jpg
coconut_27.jpg
coconut_28.jpg
coconut_29.jpg
coconut_3.jpg
coconut_30.jpg
coconut_31.jpg
coconut_32.

### Normalization - Min-Max normalization

In [3]:
def normalize_images(directory):
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):
                filepath = subdir + os.sep + file    
                img = cv2.imread(filepath)
                img = img.astype('float32')
                min_val = np.min(img)
                max_val = np.max(img)
                normalized_img = (img - min_val) / (max_val - min_val)
                cv2.imwrite(filepath, normalized_img*255)


In [28]:
normalize_images(r"C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split\train")

### Image Enhancement -  histogram_equalization

In [4]:
def histogram_equalization(directory):
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            print(file)
            if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):
                filepath = subdir + os.sep + file    
                img = cv2.imread(filepath)
                
                img = img.astype('int')
                hist, bins = np.histogram(img.flatten(), 256, [0, 256])
                cdf = hist.cumsum()
                cdf_m = np.ma.masked_equal(cdf, 0)
                cdf_m = (cdf_m - cdf_m.min()) * 255 / (cdf_m.max() - cdf_m.min())
                cdf = np.ma.filled(cdf_m, 0).astype('uint8')
                img_eq = cdf[img]
                cv2.imwrite(filepath, img_eq)


In [31]:
histogram_equalization(r"C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split\train")

banana_1.jpg
banana_10.jpg
banana_13.jpeg
banana_15.png
banana_16.jpg
banana_18.jpg
banana_2.png
banana_22.jpg
banana_23.jpg
banana_25.jpg
banana_26.jpeg
banana_29.jpg
banana_3.jpg
banana_30.jpg
banana_32.jpg
banana_36.jpg
banana_38.jpg
banana_39.jpeg
banana_4.jpg
banana_41.png
banana_42.jpg
banana_43.jpg
banana_44.jpg
banana_45.jpg
banana_46.jpg
banana_47.jpg
banana_48.jpg
banana_5.jpg
banana_50.jpg
banana_51.jpg
banana_53.jpg
banana_54.jpg
banana_55.jpg
banana_56.jpg
banana_59.jpg
banana_6.jpg
banana_61.jpg
banana_62.jpg
banana_64.jpg
banana_65.jpg
banana_66.jpg
banana_68.jpg
banana_69.jpg
banana_7.jpg
banana_70.jpg
banana_72.jpg
banana_74.jpg
banana_75.jpg
banana_76.jpg
banana_78.jpg
banana_8.jpg
banana_80.jpg
banana_87.jpg
banana_9.jpg
coconut_0.jpeg
coconut_11.jpg
coconut_12.jpeg
coconut_17.jpg
coconut_18.jpg
coconut_19.jpg
coconut_2.jpg
coconut_23.jpg
coconut_25.jpg
coconut_26.jpg
coconut_27.jpg
coconut_28.jpg
coconut_29.jpg
coconut_3.jpg
coconut_30.jpg
coconut_31.jpg
coconut_32.

# Image Augmentation

In [60]:
# datagen = ImageDataGenerator(
#         rotation_range=60,
#         width_shift_range=0.3,
#         height_shift_range=0.3,
#         shear_range=0.3,
#         zoom_range=0.3,
#         horizontal_flip=True,
#         fill_mode='constant')
datagen = ImageDataGenerator(
        rotation_range=25,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.1,
        zoom_range=0.15,
        horizontal_flip=True,
        fill_mode='nearest')

In [42]:
def augment(folder_dir, new_folder, monocot):
    os.listdir(folder_dir)
    for file in os.listdir(folder_dir):
        f_img = folder_dir+"/"+file
        img = Image.open(f_img)
        print(type(img))
        x = img_to_array(img) 
        x = x.reshape((1,) + x.shape)
        i = 0
        for batch in datagen.flow(x, batch_size=1,save_to_dir=new_folder, save_prefix=monocot):
            i += 1
            if i > 10:
                break  # otherwise the generator would loop indefinitely

In [55]:
plant_names=["banana", "coconut", "maize", "wheat", "sugarcane"]

In [61]:
for i in plant_names:
#     folder_dir = r'F:\DUMP\Dataset - Copy\\'+ monocot
#     new_folder=r'F:\DUMP\Datasetupdated\wheat\\'+ monocot
    folder_dir=r"C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split\train\\"+i
    new_floder=r"C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split\Final Train2\\"+i
    augment(folder_dir, new_floder, i)

<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageF

<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.JpegImageFile'>
<class 'PIL.JpegImagePlugin.Jpeg

# Resize Images 

In [8]:
def resize_images(directory):
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            print(file)
            if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):
                filepath = subdir + os.sep + file    
                img = cv2.imread(filepath)
                dst = cv2.resize(img, (224,224), interpolation=cv2.INTER_LANCZOS4)
                cv2.imwrite(filepath, dst)

In [67]:
resize_images(r"C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split\Final Train2 - resized")

banana_0_1049.png
banana_0_1101.png
banana_0_1102.png
banana_0_1104.png
banana_0_1107.png
banana_0_1134.png
banana_0_1137.png
banana_0_1145.png
banana_0_1166.png
banana_0_118.png
banana_0_1186.png
banana_0_1197.png
banana_0_12.png
banana_0_1211.png
banana_0_1234.png
banana_0_1254.png
banana_0_1258.png
banana_0_126.png
banana_0_1271.png
banana_0_1302.png
banana_0_1319.png
banana_0_1325.png
banana_0_1330.png
banana_0_1335.png
banana_0_1342.png
banana_0_1343.png
banana_0_1347.png
banana_0_1350.png
banana_0_1351.png
banana_0_1365.png
banana_0_1378.png
banana_0_1411.png
banana_0_1418.png
banana_0_1430.png
banana_0_145.png
banana_0_1455.png
banana_0_146.png
banana_0_1465.png
banana_0_1475.png
banana_0_1492.png
banana_0_1501.png
banana_0_1516.png
banana_0_1526.png
banana_0_1538.png
banana_0_1539.png
banana_0_1540.png
banana_0_1541.png
banana_0_1552.png
banana_0_1636.png
banana_0_1649.png
banana_0_1650.png
banana_0_1705.png
banana_0_1706.png
banana_0_1720.png
banana_0_1721.png
banana_0_1746.pn

banana_0_7995.png
banana_0_8014.png
banana_0_8043.png
banana_0_8068.png
banana_0_8077.png
banana_0_8127.png
banana_0_8145.png
banana_0_8151.png
banana_0_8153.png
banana_0_8155.png
banana_0_8186.png
banana_0_8203.png
banana_0_8205.png
banana_0_8222.png
banana_0_8223.png
banana_0_8228.png
banana_0_8265.png
banana_0_829.png
banana_0_8291.png
banana_0_8302.png
banana_0_831.png
banana_0_8319.png
banana_0_8324.png
banana_0_839.png
banana_0_8390.png
banana_0_84.png
banana_0_8426.png
banana_0_8451.png
banana_0_8458.png
banana_0_846.png
banana_0_8486.png
banana_0_8535.png
banana_0_860.png
banana_0_8606.png
banana_0_8613.png
banana_0_8638.png
banana_0_865.png
banana_0_8669.png
banana_0_8678.png
banana_0_8688.png
banana_0_8692.png
banana_0_8716.png
banana_0_8731.png
banana_0_8735.png
banana_0_8760.png
banana_0_8786.png
banana_0_8795.png
banana_0_8803.png
banana_0_8817.png
banana_0_8847.png
banana_0_8852.png
banana_0_8859.png
banana_0_8865.png
banana_0_8874.png
banana_0_8879.png
banana_0_8883.png


coconut_0_8783.png
coconut_0_8793.png
coconut_0_8798.png
coconut_0_8816.png
coconut_0_8823.png
coconut_0_8829.png
coconut_0_8857.png
coconut_0_886.png
coconut_0_8911.png
coconut_0_8915.png
coconut_0_8942.png
coconut_0_8963.png
coconut_0_8968.png
coconut_0_8975.png
coconut_0_9020.png
coconut_0_9035.png
coconut_0_9048.png
coconut_0_9050.png
coconut_0_9059.png
coconut_0_9090.png
coconut_0_9166.png
coconut_0_9171.png
coconut_0_9184.png
coconut_0_9188.png
coconut_0_9217.png
coconut_0_9292.png
coconut_0_9307.png
coconut_0_9322.png
coconut_0_9329.png
coconut_0_934.png
coconut_0_9345.png
coconut_0_9401.png
coconut_0_9461.png
coconut_0_9466.png
coconut_0_9491.png
coconut_0_9509.png
coconut_0_9548.png
coconut_0_9563.png
coconut_0_9632.png
coconut_0_9705.png
coconut_0_9706.png
coconut_0_9712.png
coconut_0_9719.png
coconut_0_9722.png
coconut_0_9752.png
coconut_0_9754.png
coconut_0_9776.png
coconut_0_978.png
coconut_0_9896.png
coconut_0_9938.png
coconut_0_9973.png
coconut_0_9991.png
maize_0_100.png

maize_0_624.png
maize_0_6245.png
maize_0_6302.png
maize_0_6322.png
maize_0_6325.png
maize_0_6327.png
maize_0_6364.png
maize_0_6369.png
maize_0_6383.png
maize_0_6387.png
maize_0_639.png
maize_0_6403.png
maize_0_6426.png
maize_0_6437.png
maize_0_6440.png
maize_0_6445.png
maize_0_6450.png
maize_0_6465.png
maize_0_6466.png
maize_0_648.png
maize_0_6506.png
maize_0_6512.png
maize_0_6513.png
maize_0_6552.png
maize_0_6581.png
maize_0_6585.png
maize_0_6605.png
maize_0_6610.png
maize_0_6624.png
maize_0_6648.png
maize_0_6653.png
maize_0_6662.png
maize_0_6663.png
maize_0_6668.png
maize_0_6678.png
maize_0_6686.png
maize_0_6689.png
maize_0_6697.png
maize_0_6698.png
maize_0_6704.png
maize_0_6713.png
maize_0_6715.png
maize_0_6719.png
maize_0_673.png
maize_0_6735.png
maize_0_6741.png
maize_0_675.png
maize_0_6766.png
maize_0_6795.png
maize_0_68.png
maize_0_6807.png
maize_0_681.png
maize_0_6811.png
maize_0_6818.png
maize_0_6819.png
maize_0_6821.png
maize_0_6822.png
maize_0_6823.png
maize_0_6833.png
maize

sugarcane_0_3410.png
sugarcane_0_3412.png
sugarcane_0_3415.png
sugarcane_0_3417.png
sugarcane_0_3425.png
sugarcane_0_3433.png
sugarcane_0_3476.png
sugarcane_0_3485.png
sugarcane_0_3494.png
sugarcane_0_3509.png
sugarcane_0_3540.png
sugarcane_0_3574.png
sugarcane_0_3614.png
sugarcane_0_3628.png
sugarcane_0_3633.png
sugarcane_0_3656.png
sugarcane_0_368.png
sugarcane_0_3717.png
sugarcane_0_3748.png
sugarcane_0_3755.png
sugarcane_0_3831.png
sugarcane_0_3834.png
sugarcane_0_3846.png
sugarcane_0_3854.png
sugarcane_0_3871.png
sugarcane_0_3938.png
sugarcane_0_3973.png
sugarcane_0_3988.png
sugarcane_0_3994.png
sugarcane_0_4023.png
sugarcane_0_4034.png
sugarcane_0_4056.png
sugarcane_0_407.png
sugarcane_0_4102.png
sugarcane_0_4105.png
sugarcane_0_4111.png
sugarcane_0_4116.png
sugarcane_0_4123.png
sugarcane_0_4140.png
sugarcane_0_4143.png
sugarcane_0_4150.png
sugarcane_0_4162.png
sugarcane_0_4169.png
sugarcane_0_4174.png
sugarcane_0_4180.png
sugarcane_0_4185.png
sugarcane_0_4218.png
sugarcane_0_424

sugarcane_0_9703.png
sugarcane_0_9705.png
sugarcane_0_9733.png
sugarcane_0_9735.png
sugarcane_0_9741.png
sugarcane_0_9764.png
sugarcane_0_9768.png
sugarcane_0_9808.png
sugarcane_0_9837.png
sugarcane_0_9849.png
sugarcane_0_9875.png
sugarcane_0_9894.png
sugarcane_0_9895.png
sugarcane_0_993.png
sugarcane_0_994.png
sugarcane_0_9940.png
sugarcane_0_9944.png
sugarcane_0_9959.png
sugarcane_0_9960.png
sugarcane_0_9972.png
sugarcane_0_9988.png
sugarcane_0_9993.png
wheat_0_0.png
wheat_0_100.png
wheat_0_1007.png
wheat_0_1011.png
wheat_0_1042.png
wheat_0_1055.png
wheat_0_1057.png
wheat_0_106.png
wheat_0_1097.png
wheat_0_1110.png
wheat_0_1118.png
wheat_0_1122.png
wheat_0_1126.png
wheat_0_113.png
wheat_0_1144.png
wheat_0_1165.png
wheat_0_1202.png
wheat_0_1214.png
wheat_0_123.png
wheat_0_1249.png
wheat_0_1260.png
wheat_0_1292.png
wheat_0_1296.png
wheat_0_1298.png
wheat_0_1305.png
wheat_0_1318.png
wheat_0_132.png
wheat_0_1328.png
wheat_0_1330.png
wheat_0_1355.png
wheat_0_1373.png
wheat_0_1378.png
whea

wheat_0_5594.png
wheat_0_5599.png
wheat_0_5606.png
wheat_0_5611.png
wheat_0_5618.png
wheat_0_5619.png
wheat_0_562.png
wheat_0_5626.png
wheat_0_5639.png
wheat_0_5640.png
wheat_0_5666.png
wheat_0_5672.png
wheat_0_5677.png
wheat_0_5693.png
wheat_0_5697.png
wheat_0_5699.png
wheat_0_5704.png
wheat_0_5705.png
wheat_0_571.png
wheat_0_5711.png
wheat_0_5717.png
wheat_0_572.png
wheat_0_5727.png
wheat_0_5731.png
wheat_0_5746.png
wheat_0_5750.png
wheat_0_5756.png
wheat_0_5769.png
wheat_0_5781.png
wheat_0_5799.png
wheat_0_580.png
wheat_0_5802.png
wheat_0_5805.png
wheat_0_5811.png
wheat_0_5812.png
wheat_0_5838.png
wheat_0_5859.png
wheat_0_5898.png
wheat_0_5911.png
wheat_0_5919.png
wheat_0_5926.png
wheat_0_5928.png
wheat_0_5937.png
wheat_0_5943.png
wheat_0_5949.png
wheat_0_5955.png
wheat_0_5956.png
wheat_0_5983.png
wheat_0_5992.png
wheat_0_5994.png
wheat_0_60.png
wheat_0_6005.png
wheat_0_6007.png
wheat_0_6011.png
wheat_0_6017.png
wheat_0_6024.png
wheat_0_6027.png
wheat_0_6032.png
wheat_0_6034.png
whe

## Dataset Balancing

In [2]:
import os
import random
import shutil
# Banana
folder_path = r'C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split\Final Train2 - resized\banana'
output_folder_path = r'C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split\Balanced\banana'
n = 385  # number of random image to select

if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

files = os.listdir(folder_path)
selected_files = random.sample(files, n)

for file in selected_files:
    file_path = os.path.join(folder_path, file)
    output_file_path = os.path.join(output_folder_path, file)
    shutil.copy(file_path, output_file_path)

# Coconut
folder_path = r'C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split\Final Train2 - resized\coconut'
output_folder_path = r'C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split\Balanced\coconut'
n = 385  # number of random image to select

if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

files = os.listdir(folder_path)
selected_files = random.sample(files, n)

for file in selected_files:
    file_path = os.path.join(folder_path, file)
    output_file_path = os.path.join(output_folder_path, file)
    shutil.copy(file_path, output_file_path)

# Maize
folder_path = r'C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split\Final Train2 - resized\maize'
output_folder_path = r'C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split\Balanced\maize'
n = 385  # number of random image to select

if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

files = os.listdir(folder_path)
selected_files = random.sample(files, n)

for file in selected_files:
    file_path = os.path.join(folder_path, file)
    output_file_path = os.path.join(output_folder_path, file)
    shutil.copy(file_path, output_file_path)

# Sugarcane
folder_path = r'C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split\Final Train2 - resized\sugarcane'
output_folder_path = r'C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split\Balanced\sugarcane'
n = 385  # number of random image to select

if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

files = os.listdir(folder_path)
selected_files = random.sample(files, n)

for file in selected_files:
    file_path = os.path.join(folder_path, file)
    output_file_path = os.path.join(output_folder_path, file)
    shutil.copy(file_path, output_file_path)

# Wheat
folder_path = r'C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split\Final Train2 - resized\wheat'
output_folder_path = r'C:\Users\GREESHMANTH\Desktop\Major Project\Code\Data_split\Balanced\wheat'
n = 385  # number of random image to select

if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

files = os.listdir(folder_path)
selected_files = random.sample(files, n)

for file in selected_files:
    file_path = os.path.join(folder_path, file)
    output_file_path = os.path.join(output_folder_path, file)
    shutil.copy(file_path, output_file_path)
